<a href="https://colab.research.google.com/github/YUHYUNSEONG/YHS/blob/master/bioinfoproject_YHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2023 생물정보학 실습 프로젝트 유현성

자유 주제 예시에서 mission 3 확장 부분을 진행할 예정입니다. 
2j3에 대해서 같은 분석을 반복해 볼 계획입니다. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 76 (delta 26), reused 59 (delta 15), pack-reused 0
Unpacking objects: 100% (76/76), 318.14 KiB | 1.38 MiB/s, done.
./
./root/
./root/.condarc
./root/.bashrc.biolab
./root/.vimrc
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.tmux.conf
./root/.profile
--2023-06-01 12:25:42--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73134376 (70M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  69.75M   133MB/s    in 0.5s    

2023-06-01 12:25:42 (133 MB/

In [3]:
!ls -al drive/MyDrive/binfo*

total 9095569
-rw------- 1 root root     149732 May 31 04:57 '3주차 최종.svg'
-rw------- 1 root root  959059112 Jun  1 07:55  CLIP-2J3.fastq.gz
-rw------- 1 root root 1370036258 May 11 07:59  CLIP-35L33G.bam
-rw------- 1 root root    3118336 May 11 07:59  CLIP-35L33G.bam.bai
-rw------- 1 root root       8704 May 18 08:07  CLIP-let7d.bam
-rw------- 1 root root      20809 May 18 08:13  CLIP-let7d.pileup
-rw------- 1 root root       7238 May 18 08:07  CLIP-let7f-1.bam
-rw------- 1 root root      17802 May 18 08:13  CLIP-let7f-1.pileup
-rw------- 1 root root       7113 May 31 01:14  CLIP-let7g.bam
-rw------- 1 root root      14561 May 31 01:14  CLIP-let7g-gene.pileup
-rw------- 1 root root    2685065 May 31 01:14  CLIP-let7g.pileup
-rw------- 1 root root       7113 May 31 01:14  CLIP-let7g-sorted.bam
-rw------- 1 root root      52392 May 31 01:14  CLIP-let7g-sorted.bam.bai
-rw------- 1 root root  572853320 May 25 13:07  filtered-RPF-siLuc.bam
-rw------- 1 root root   28109361 May 25 13:33

In [4]:
!conda install -y subread

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0



## Package Plan ##

  environment location: /root/conda

  added / updated specs:
    - subread


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forg

In [5]:
%cd /content/drive/MyDrive/binfo1-work

/content/drive/MyDrive/binfo1-work


 유전자 annotation에 대한 정보   mirlet7g 의 annotation 정보 찾기

In [6]:
!grep -i mirlet7g gencode.gtf

chr9	ENSEMBL	gene	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; gene_type "miRNA"; gene_name "Mirlet7g"; level 3; mgi_id "MGI:2676800";
chr9	ENSEMBL	transcript	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";
chr9	ENSEMBL	exon	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; exon_number 1; exon_id "ENSMUSE00000522665.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";


In [7]:
!ls

'3주차 최종.svg'
 CLIP-2J3.fastq.gz
 CLIP-35L33G.bam
 CLIP-35L33G.bam.bai
 CLIP-let7d.bam
 CLIP-let7d.pileup
 CLIP-let7f-1.bam
 CLIP-let7f-1.pileup
 CLIP-let7g.bam
 CLIP-let7g-gene.pileup
 CLIP-let7g.pileup
 CLIP-let7g-sorted.bam
 CLIP-let7g-sorted.bam.bai
 filtered-RPF-siLuc.bam
 fivepcounts-filtered-RPF-siLuc.txt
 fivepcounts-RPF-siLuc.bed
 gencode-exons-containing-startcodon.bed
 gencode.gtf
 gencode-plusexon.gtf
 gencode-start.gtf
 hgt_genome_28809_5ed4f0.pdf
 Mus_musculus_C57BL6J_chromosome9.fasta
 Mus_musculus_C57BL6J_chromosome9.fasta.amb
 Mus_musculus_C57BL6J_chromosome9.fasta.ann
 Mus_musculus_C57BL6J_chromosome9.fasta.bwt
 Mus_musculus_C57BL6J_chromosome9.fasta.pac
 Mus_musculus_C57BL6J_chromosome9.fasta.sa
 page_1.png
 read-counts.txt
 read-counts.txt.summary
 RNA-control.bam
 RNA-control.bam.bai
 RNA-siLin28a.bam
 RNA-siLin28a.bam.bai
 RNA-siLuc.bam
 RNA-siLuc.bam.bai
 RPF-siLin28a.bam
 RPF-siLin28a.bam.bai
 RPF-siLuc.bam
 RPF-siLuc.bam.bai
'shannon entropy, bam file.svg'


In [8]:
!sudo apt-get install bwa

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  samtools
The following NEW packages will be installed:
  bwa
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 204 kB of archives.
After this operation, 482 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 bwa amd64 0.7.17-4 [204 kB]
Fetched 204 kB in 1s (218 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package bwa.
(Reading database ... 122545 files and dire

In [ ]:
!bwa index -a is /content/drive/MyDrive/binfo1-work/Mus_musculus_C57BL6J_chromosome9.fasta


[bwa_index] Pack FASTA... 1.62 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 119.21 seconds elapse.
[bwa_index] Update BWT... 1.18 sec
[bwa_index] Pack forward-only FASTA... 1.26 sec
[bwa_index] Construct SA from BWT and Occ... 50.38 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index -a is /content/drive/MyDrive/binfo1-work/Mus_musculus_C57BL6J_chromosome9.fasta
[main] Real time: 185.184 sec; CPU: 173.676 sec


In [9]:
!gunzip CLIP-2J3.fastq.gz

In [11]:
!less CLIP-2J3.fastq

@SRR458759.1.1 GA01_0016_FC:2:1:2345:1035 length=78
GAAGCNAAGATTGGAGTGTTCGATTCTGTAGGCACCATCAATTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAACA
+SRR458759.1.1 GA01_0016_FC:2:1:2345:1035 length=78
??????????????????????????????????????????????????????????????????????????????
@SRR458759.2.1 GA01_0016_FC:2:1:2582:1035 length=78
GTGGTNGGACTGCTCAGGGGAGTGGTGCAGTGTGATTCCCGCCGGTCTGTAGGCACATCAATTCGTATGCCGTCTTCT
+SRR458759.2.1 GA01_0016_FC:2:1:2582:1035 length=78
??????????????????????????????????????????????????????????????????????????????
@SRR458759.3.1 GA01_0016_FC:2:1:4316:1035 length=78
ATACCNCCCTGAACGCGCCCGATCTCGTCTGATCTCGGAAGCTAAGCAGGGTCGGGCCTGGTTAGTACTTGGATGGGA
+SRR458759.3.1 GA01_0016_FC:2:1:4316:1035 length=78
??????????????????????????????????????????????????????????????????????????????
@SRR458759.4.1 GA01_0016_FC:2:1:4675:1034 length=78
AGCTTNCGGGAAACCAAAGTCTTTGGGTTCCGGGGGGAGCTGTAGGCACCATCAATTCGTATGCCGTCTTCTGCTTGA
+SRR458759.4.1 GA01_0016_FC:2:1:4675:1034 length=78
???????????????????????????????

In [12]:
!ls -al

total 17322978
-rw------- 1 root root     149732 May 31 04:57 '3주차 최종.svg'
-rw------- 1 root root 9383925840 Jun  1 07:55  CLIP-2J3.fastq
-rw------- 1 root root 1370036258 May 11 07:59  CLIP-35L33G.bam
-rw------- 1 root root    3118336 May 11 07:59  CLIP-35L33G.bam.bai
-rw------- 1 root root       8704 May 18 08:07  CLIP-let7d.bam
-rw------- 1 root root      20809 May 18 08:13  CLIP-let7d.pileup
-rw------- 1 root root       7238 May 18 08:07  CLIP-let7f-1.bam
-rw------- 1 root root      17802 May 18 08:13  CLIP-let7f-1.pileup
-rw------- 1 root root       7113 May 31 01:14  CLIP-let7g.bam
-rw------- 1 root root      14561 May 31 01:14  CLIP-let7g-gene.pileup
-rw------- 1 root root    2685065 May 31 01:14  CLIP-let7g.pileup
-rw------- 1 root root       7113 May 31 01:14  CLIP-let7g-sorted.bam
-rw------- 1 root root      52392 May 31 01:14  CLIP-let7g-sorted.bam.bai
-rw------- 1 root root  572853320 May 25 13:07  filtered-RPF-siLuc.bam
-rw------- 1 root root   28109361 May 25 13:33  

In [ ]:
!bwa mem Mus_musculus_C57BL6J_chromosome9.fasta LIN28A_CLIP-2J3mAb.fastq.gz | less

:

KeyboardInterrupt: ignored